<a href="https://colab.research.google.com/github/Teja220/Tata-Communications-Cifar-Image-Classification/blob/main/Bhimavarapu_Saiteja(21BDS0277).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saitejaredyy (saitejaredyy-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import wandb
wandb.init(project="cifar10-image-classification")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Data augmentation and normalization for training
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:01<00:00, 104MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#We will count the number of training images and test images, this is essential to understand the dataset
num_train_images = len(trainset)
num_test_images = len(testset)
total_images = num_train_images + num_test_images

print(f"Number of training images: {num_train_images}")
print(f"Number of test images: {num_test_images}")
print(f"Total number of images: {total_images}")

Number of training images: 50000
Number of test images: 10000
Total number of images: 60000


In [ ]:
#Further after checking the data set we will see if there are any corrupted samples there in the dataset or not, it's the basic step we need to do in any ML process to find the outliers in dataset
def check_dataset(dataset):
    corrupted = 0
    for i in range(len(dataset)):
        try:
            image, label = dataset[i]
        except Exception as e:
            print(f"Error in sample {i}: {e}")
            corrupted += 1
    if corrupted == 0:
        print("No corrupted samples found!")
    else:
        print(f"Found {corrupted} corrupted samples.")

check_dataset(trainset)


No corrupted samples found!


1. First, we defined a simple CNN architecture to know how the model is performing and then we can fine tune it further in the notebook
2. Two convolutional layers for feature extraction
3. Max-pooling for dimensionality reduction
4. Fully connected layers for classification
5. ReLU activation for non-linearity

In [ ]:
from torch import nn, optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()


1. Cross-Entropy Loss is commonly used for multi-class classification problems.   
2. Adam is an adaptive learning rate optimization algorithm.
3. lr is the learning rate, lr=0.001 is the default lr for adam optimizer




In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Defining the function to train the required number of epochs
def train_model(epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0

        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total_correct += (predicted == labels).sum().item()

        # Help's us to Calculate metrics
        epoch_loss = running_loss / len(trainloader)
        epoch_accuracy = total_correct / len(trainset)

        # Log metrics to W&B
        wandb.log({
            'Epoch': epoch + 1,
            'Training Loss': epoch_loss,
            'Training Accuracy': epoch_accuracy
        })

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Recall the function to start training
train_model(epochs=5)


Epoch 1, Loss: 0.9590, Accuracy: 0.6615
Epoch 2, Loss: 0.8038, Accuracy: 0.7189
Epoch 3, Loss: 0.6876, Accuracy: 0.7582
Epoch 4, Loss: 0.5814, Accuracy: 0.7970
Epoch 5, Loss: 0.4856, Accuracy: 0.8286


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saitejaredyy (saitejaredyy-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

#Hyper parameter Tuning - 1

* Since we got the initial accuracy of 0.82 further by fine tuning we should be
to achieve 90 or 95.
* Hyper parameter Tuning - 1, key changes:

1. Optimizer: SGD
2. Learning rate: 0.01
3. Batch size: 64



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
import wandb

# Initialize W&B properly with the correct run name
wandb.init(project="cifar10-image-classification", name="Hyperparameter Tuning 1", reinit=True)

# Load CIFAR-10 dataset with batch size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initializing model, criterion, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop with W&B logging
def train_model(epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0

        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total_correct += (predicted == labels).sum().item()

        # Calculate metrics
        epoch_loss = running_loss / len(trainloader)
        epoch_accuracy = total_correct / len(trainset)

        # Log metrics to W&B
        wandb.log({
            'Epoch': epoch + 1,
            'Training Loss': epoch_loss,
            'Training Accuracy': epoch_accuracy
        })

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Properly close this W&B run
    wandb.finish()

# Start training for Hyperparameter Tuning 1
train_model(epochs=5)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


100%|██████████| 170M/170M [00:03<00:00, 43.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1, Loss: 2.0680, Accuracy: 0.2695
Epoch 2, Loss: 1.6774, Accuracy: 0.4062
Epoch 3, Loss: 1.4857, Accuracy: 0.4702
Epoch 4, Loss: 1.3794, Accuracy: 0.5057
Epoch 5, Loss: 1.2906, Accuracy: 0.5397


Epoch,▁▃▅▆█
Training Accuracy,▁▅▆▇█
Training Loss,█▄▃▂▁
Epoch,5
Training Accuracy,0.53974
Training Loss,1.29065


#Hyperparameter Tuning - 2

* After seeing the results by changing the optimizer from ADAM to SGD, the accuracy dropped drastically

* We can consider ruling out the SGD optimizer

* For the next run we will implement the following changes:

1. Optimizer: Adam
2. Learning rate: 0.001
3. Batch size: 128

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
import wandb

# Initialize W&B for Hyperparameter Tuning 2
wandb.init(project="cifar10-image-classification", name="Hyperparameter Tuning 2", reinit=True)

# Load CIFAR-10 dataset with batch size = 128
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initializing model, criterion, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with W&B logging
def train_model(epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0

        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total_correct += (predicted == labels).sum().item()

        # Calculate metrics
        epoch_loss = running_loss / len(trainloader)
        epoch_accuracy = total_correct / len(trainset)

        # Log metrics to W&B
        wandb.log({
            'Epoch': epoch + 1,
            'Training Loss': epoch_loss,
            'Training Accuracy': epoch_accuracy
        })

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Properly close this W&B run
    wandb.finish()

# Start training for Hyperparameter Tuning 2
train_model(epochs=5)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 1.4217, Accuracy: 0.4878
Epoch 2, Loss: 1.0472, Accuracy: 0.6329
Epoch 3, Loss: 0.8809, Accuracy: 0.6930
Epoch 4, Loss: 0.7738, Accuracy: 0.7292
Epoch 5, Loss: 0.6818, Accuracy: 0.7638


Epoch,▁▃▅▆█
Training Accuracy,▁▅▆▇█
Training Loss,█▄▃▂▁
Epoch,5
Training Accuracy,0.76382
Training Loss,0.68177


#Hyperparameter Tuning - 3

* So after using the ADAM optimizer and improved batch size to 128, we have improved the accuracy from 0.54(approx) to ~0.77. Now we will make the folllowing changes:

1. Optimizer: Adam
2. Learning rate: 0.0005
3. Batch size: 128

We have just adjusted the learnning rate to 0.0005 from 0.001

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
import wandb

# Initialize W&B for Hyperparameter Tuning 3
wandb.init(project="cifar10-image-classification", name="Hyperparameter Tuning 3", reinit=True)

# Load CIFAR-10 dataset with batch size = 128
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initializing model, criterion, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training loop with W&B logging
def train_model(epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0

        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total_correct += (predicted == labels).sum().item()

        # Calculate metrics
        epoch_loss = running_loss / len(trainloader)
        epoch_accuracy = total_correct / len(trainset)

        # Log metrics to W&B
        wandb.log({
            'Epoch': epoch + 1,
            'Training Loss': epoch_loss,
            'Training Accuracy': epoch_accuracy
        })

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Properly close this W&B run
    wandb.finish()

# Start training for Hyperparameter Tuning 3
train_model(epochs=5)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 1.5351, Accuracy: 0.4512
Epoch 2, Loss: 1.2030, Accuracy: 0.5776
Epoch 3, Loss: 1.0480, Accuracy: 0.6321
Epoch 4, Loss: 0.9461, Accuracy: 0.6706
Epoch 5, Loss: 0.8701, Accuracy: 0.6953


Epoch,▁▃▅▆█
Training Accuracy,▁▅▆▇█
Training Loss,█▅▃▂▁
Epoch,5
Training Accuracy,0.69534
Training Loss,0.8701


#Hyperparameter Tuning - 4

* Doing this we have gain lost the accuracy falling back to 0.69 from ~0.77, now we will increase the learning rate and the number of epochs:

1. Optimizer: Adam
2. Learning rate: 0.0008
3. Batch size: 128
4. Epochs: 10

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
import wandb

# Initialize W&B for Hyperparameter Tuning 4
wandb.init(project="cifar10-image-classification", name="Hyperparameter Tuning 4", reinit=True)

# Load CIFAR-10 dataset with batch size = 128
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initializing model, criterion, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008)

# Training loop with W&B logging
def train_model(epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0

        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total_correct += (predicted == labels).sum().item()

        # Calculate metrics
        epoch_loss = running_loss / len(trainloader)
        epoch_accuracy = total_correct / len(trainset)

        # Log metrics to W&B
        wandb.log({
            'Epoch': epoch + 1,
            'Training Loss': epoch_loss,
            'Training Accuracy': epoch_accuracy
        })

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Properly close this W&B run
    wandb.finish()

# Start training for Hyperparameter Tuning 4
train_model(epochs=10)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 1.4752, Accuracy: 0.4712
Epoch 2, Loss: 1.1254, Accuracy: 0.6009
Epoch 3, Loss: 0.9603, Accuracy: 0.6612
Epoch 4, Loss: 0.8427, Accuracy: 0.7051
Epoch 5, Loss: 0.7523, Accuracy: 0.7386
Epoch 6, Loss: 0.6679, Accuracy: 0.7678
Epoch 7, Loss: 0.5916, Accuracy: 0.7932
Epoch 8, Loss: 0.5249, Accuracy: 0.8185
Epoch 9, Loss: 0.4538, Accuracy: 0.8428
Epoch 10, Loss: 0.3918, Accuracy: 0.8651


Epoch,▁▂▃▃▄▅▆▆▇█
Training Accuracy,▁▃▄▅▆▆▇▇██
Training Loss,█▆▅▄▃▃▂▂▁▁
Epoch,10
Training Accuracy,0.86506
Training Loss,0.39184


#Hyperparameter Tuning - 5

* We got a good accuracy of 0.865 from the previous run 0.69, if we run 25 epochs we can easily reach 0.90 accuracy.

* This experiment introduces model architecture improvements with:
* Increased convolutional filters for better feature extraction.
* Reduced dropout (p=0.1) for effective learning without overfitting.

* We have implemented, the following:

1. Increased filters: 32 → 64, 64 → 128
2. Dropout: p=0.1
3. Learning rate: 0.0008
4. Batch size: 128
5. Epochs: 10


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
import wandb

# Initialize W&B for Hyperparameter Tuning 6
wandb.init(project="cifar10-image-classification", name="Hyperparameter Tuning 5", reinit=True)

# Load CIFAR-10 dataset with batch size = 128
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

# Define the model with more filters and reduced dropout
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # Increased filters
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Increased filters
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.1)  # Reduced dropout
        self.fc1 = nn.Linear(128 * 8 * 8, 256)  # Increased layer size
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.dropout(self.relu(self.fc1(x)))  # Apply dropout
        x = self.fc2(x)
        return x

# Initializing model, criterion, and optimizer
model = EnhancedCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008)

# Training loop with W&B logging
def train_model(epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_correct = 0

        for inputs, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total_correct += (predicted == labels).sum().item()

        # Calculate metrics
        epoch_loss = running_loss / len(trainloader)
        epoch_accuracy = total_correct / len(trainset)

        # Log metrics to W&B
        wandb.log({
            'Epoch': epoch + 1,
            'Training Loss': epoch_loss,
            'Training Accuracy': epoch_accuracy
        })

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Properly close this W&B run
    wandb.finish()

# Start training for Hyperparameter Tuning 6
train_model(epochs=10)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 1.3490, Accuracy: 0.5162
Epoch 2, Loss: 0.9617, Accuracy: 0.6604
Epoch 3, Loss: 0.7912, Accuracy: 0.7223
Epoch 4, Loss: 0.6595, Accuracy: 0.7685
Epoch 5, Loss: 0.5357, Accuracy: 0.8124
Epoch 6, Loss: 0.4238, Accuracy: 0.8527
Epoch 7, Loss: 0.3227, Accuracy: 0.8878
Epoch 8, Loss: 0.2315, Accuracy: 0.9204
Epoch 9, Loss: 0.1702, Accuracy: 0.9433
Epoch 10, Loss: 0.1317, Accuracy: 0.9564


Epoch,▁▂▃▃▄▅▆▆▇█
Training Accuracy,▁▃▄▅▆▆▇▇██
Training Loss,█▆▅▄▃▃▂▂▁▁
Epoch,10
Training Accuracy,0.95638
Training Loss,0.13169


#Key Findings:

1. Adam's adaptive learning technique beat SGD, particularly at modest learning rates.
2. Lowering the learning rate too much negatively impacted performance, while balancing it (~0.0008) improved accuracy.
3. Batch size: A batch size of 128 achieved a decent blend of gradient stability and speedier convergence.
4. Enhanced architecture: Increasing convolutional filters and decreasing dropout resulted in much higher accuracy, demonstrating the importance of model capacity for feature extraction.

#Best Configuration:

1. Optimizer: Adam
2. Learning rate: 0.0008
3. Batch size: 128
4. Dropout: 0.1
5. Epochs: 10
6. Enhanced convolutional filters (64 → 128)

* This configuration produced 95.64% accuracy, demonstrating the necessity of incremental tuning and architectural enhancements for peak performance.